In [29]:
import os
import time
import csv
import shelve
import pickle
from datetime import datetime

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from matplotlib import cm

import csv
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib import cm
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GMM
from matplotlib.colors import LogNorm

np.random.seed(0)

In [30]:
from lasagne import layers
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

In [31]:
import theano
from theano import tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams
from theano.tensor.nnet.conv import conv2d
from theano.tensor.signal.downsample import max_pool_2d
print (theano.config.device) # We're using CPUs (for now)
print (theano.config.floatX) # Should be 64 bit for CPUs

cpu
float64


In [32]:
# function to load a file
def LoadFile(path):
    coordinates, faces = [], []
    with open(path) as csvfile:
        # read the file and separate image and keypoint coordinates
        facereader = csv.reader(csvfile, delimiter=',')
        for row in facereader:
            coordinates.append(row[0:-1])
            faces.append(row[-1])
        
    # first line is header, pop it out    
    coor_names, trash = coordinates.pop(0), faces.pop(0)
    # covert everything to number   
    for i in range(len(faces)):    
        faces[i] = [int(x) for x in faces[i].split(' ')]
        coordinates[i] = [float(x) if x else 0 for x in coordinates[i]]
       
    # convert to numpy array and return    
    return np.array(faces), np.array(coordinates), np.array(coor_names)

# load test data file
test_faces, test_id, trash = LoadFile('./Data/FKD_Test.csv')
print ('test data: %s, test id : %s' %(str(test_faces.shape), str(test_id.shape)))

# load training data file, and separate it into training and dev data
train_faces, train_coordinates, feature_name = LoadFile('./Data/FKD_Train.csv')
nTotal = train_faces.shape[0]
shuffle = np.random.permutation(np.arange(nTotal))
train_faces, train_coordinates = train_faces[shuffle], train_coordinates[shuffle]

# take 85% as training, 15% as dev 
nTrain = np.round(nTotal*.85)
dev_faces, dev_coordinates = train_faces[nTrain:], train_coordinates[nTrain:]
train_faces, train_coordinates = train_faces[1:nTrain], train_coordinates[1:nTrain]

print ('train data: %s, train coordinates: %s' %(str(train_faces.shape), str(train_coordinates.shape)))
print ('dev data: %s, dev coordinates: %s' %(str(dev_faces.shape), str(dev_coordinates.shape)))

test data: (1783, 9216), test id : (1783, 1)
train data: (5991, 9216), train coordinates: (5991, 30)
dev data: (1057, 9216), dev coordinates: (1057, 30)


/home/linux/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/linux/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [35]:
class MeanPatchSearching:
    # Initialize an instance of the class.
    def __init__(self, patch_size=10, search_size=10, stretch=True):
        self.patch_size = patch_size
        self.search_size = search_size
        self.isStretch = stretch
        self.patch_index = np.array([[i,j] for i in range(-patch_size, patch_size+1) 
                                           for j in range(-patch_size, patch_size+1)])
        self.search_index = np.array([[i,j] for i in range(-search_size, search_size+1) 
                                            for j in range(-search_size, search_size+1)])
    
    # train the model
    def fit(self, train_faces, train_coordinates):
        start = datetime.now()
        # stretch input if needed
        if False: #self.isStretch:
            train_faces = np.array([self._histogramStretching(x) for x in train_faces])
        # number of faces to train
        self.num_examples = train_faces.shape[0]
        print ('number of training faces: %d' %self.num_examples)
        # assuming coordinates are (x,y) pairs for each key point
        self.num_keypoints = (int)(train_coordinates.shape[1]/2)
        # image dimension
        self.ncolumn = np.sqrt(train_faces.shape[1])
        self.nrow = self.ncolumn
        
        # get patches and their centers for all keypoints
        self.patches, self.patch_centers = [], []
        for i in range(self.num_keypoints):
            # get coordinates of current keypoint
            coordinates = train_coordinates[:,i*2:(i+1)*2]
            # filter zero values (empty from file)
            picker = coordinates[:,0]>0                        
            # get patch if at least one face has this point
            if sum(picker)>0:
                # get patch for this key point
                self.patches.append(self._getPatch(train_faces[picker], coordinates[picker]))
                # get center for this keypoint
                self.patch_centers.append(np.mean(coordinates[picker], axis=0))
        
        # convert to numpy array      
        self.patches = np.array(self.patches)
        self.patch_centers = np.array(self.patch_centers)
        self.num_keypoints = self.patches.shape[0]
        self.training_time = (datetime.now()-start).total_seconds()/60.0
        print ('training patches shape: %s' %str(self.patches.shape))
        print ('training time: %.1f minutes' %self.training_time)
        # show training patches
#         self._plotPatches()
        
    # Make prediction for each test face and return coordinates.
    def predict(self, test_faces):
        start = datetime.now()
        # stretch input if needed
        if self.isStretch:
            test_faces = np.array([self._histogramStretching(x) for x in test_faces])
        self.num_predict = test_faces.shape[0]
        print ('number of predicting faces: %d' %self.num_predict)
        predictions = []
        for i in range(self.num_predict):
            if np.mod((i+1), self.num_predict/10)==0:
                print ('Complete %d%% ...' %(100.0*(i+1)/self.num_predict))
            pred = self._predictOneFace(test_faces[i])            
            predictions.append(np.reshape(pred, (1,2*self.num_keypoints))[0])        
        self.pred_coor = np.array(predictions)
        self.predict_time = (datetime.now()-start).total_seconds()/60.0
        print ('Done! - Predict time: %.1f minutes' %self.predict_time)
        return self.pred_coor
    
    # calculate total Root Mean Squared Error (RMSE)
    def RMSE(self, actual, pred=[]):
        if len(pred)==0:
            pred = self.pred_coor
        picker = actual>0        
        tRMSE = np.sqrt(np.sum((actual[picker]-pred[picker])**2)/np.sum(picker)) 
        return 'Total RMSE: %.2f, patch size: %d, search size: %d' %(tRMSE, self.patch_size, self.search_size)
    
    # save the submission file based on prediction made for test images
    def getSubmission(self, LookupTable, feature_name):
        # create a dictionary for feature name indexing
        feature_index = {x:np.where(feature_name==x)[0][0] for x in feature_name}
        lookupRow = []
        with open(LookupTable) as csvfile:
            # read the lookup file
            lookupReader = csv.reader(csvfile, delimiter=',')
            lookupRow.append(lookupReader.next())
            for row in lookupReader:
                # get the prediction based on image ID and feature name, and attach to the row
                location = self.pred_coor[int(row[1])-1, feature_index[row[2]]]
                lookupRow.append(np.append(row, location))
        lookupRow = np.array(lookupRow)
        # save row ID and location ID columns only to the submission file
        saveFile = 'submission_'+datetime.now().strftime("%Y%m%d%H%M%S")+'.csv'
        with open(saveFile, 'wb') as f:
            writer = csv.writer(f)
            writer.writerows(lookupRow[:,[0,3]])
        print ('Submission file saved as: %s' %saveFile)
        return lookupRow
        
    # get the prediction for one face
    def _predictOneFace(self, face):
        # get prediction for each keypoint available in the model
        pred_coor = []
        for gold_p, center in zip(self.patches, self.patch_centers):
            # get the candidate points based on search size
            candidates = self._getCandidates(center)
            # get a patch for each candidate point     
            pred_p = [self._getPatch([face], [x]) for x in candidates]

            # compare the patches from candidate points with gold_p
            # TODO: use better distance
            dist = [np.sum(np.abs(gold_p-x)) for x in pred_p]
            pred_coor.append(candidates[np.argmin(dist)])
        return pred_coor
    
    # get the candidate points - return the coordinates
    def _getCandidates(self, center):  
        r, c = np.round(center)
        candidates = np.array([[r+i, c+j] for i,j in self.search_index])
        # only keep those within the bound
        picker = (np.sum(candidates>=0,axis=1) + np.sum(candidates<[self.nrow,self.ncolumn],axis=1))==4        
        return candidates[picker]
    
    # get the patch for one keypoint from all faces
    def _getPatch(self, faces, keypoints):    
        patches = []
        for face, keypoint in zip(faces, keypoints):
            # get keypoint pixel row and column index
            r, c = np.round(keypoint)        
            # get indices for the patch (including self)
            neighbors = np.array([[r+i, c+j] for i,j in self.patch_index])
            if np.sum(neighbors>=0)+np.sum(neighbors<[self.nrow,self.ncolumn]) == np.prod(neighbors.shape)*2:
                patches.append(face[[r*self.ncolumn + c for r,c in neighbors]])
#             else:
#                 print ('warning - nonconforming patch')
#         print (np.array(patches).shape)
        return np.mean(patches, axis=0)

    # histogram stretching pre-processing
    def _histogramStretching(self, image):
        # a, b = min(image), max(image) 
        a, b = np.percentile(image, 5), np.percentile(image, 95)
        l, u = 0, 255
        const = 1.0*(b*l - a*u)/(b - a)
        k = 1.0*(u-l)/(b-a)
        return [k*p+const for p in image]
    
    # plot average patch from training
    def _plotPatches(self):
        n_side = 2*self.patch_size+1
        keypoints = np.reshape([x[:-2] for x in feature_name],(self.num_keypoints,2))
        plt.figure(figsize=(16, 8))
        i = 1
        for point, patch in zip(keypoints[:,0], self.patches):
            plt.subplot(3,5,i)
            plt.imshow(np.reshape(patch,(n_side,n_side)), cmap = cm.gray)
            plt.title(point)
            plt.axis('off')
            i += 1
        plt.show()
        

##### Get baseline score #####
# training
mps = MeanPatchSearching(patch_size=9, search_size=5, stretch=True)
mps.fit(train_faces[:150], train_coordinates[:150])
# predicting
predictions = mps.predict(test_faces)
mps.getSubmission('./Data/FKD_IdLookupTable.csv', feature_name)

number of training faces: 150


/home/linux/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:133: VisibleDeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future


training patches shape: (15, 361)
training time: 0.0 minutes
number of predicting faces: 1783
Done! - Predict time: 52.1 minutes


AttributeError: '_csv.reader' object has no attribute 'next'

NameError: name 'train_faces' is not defined

NameError: name 'MeanPatchSearching' is not defined